In [18]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType}
import org.apache.spark.sql.Row



val spark = SparkSession.builder
  .appName("Exercise 2")
  .master("yarn")
  .getOrCreate()

val sc = spark.sparkContext

spark = org.apache.spark.sql.SparkSession@7f0d26f6
sc = org.apache.spark.SparkContext@49ef656b


org.apache.spark.SparkContext@49ef656b

In [19]:
import org.apache.spark.sql.DataFrame
import spark.implicits._
val salesDataPath = "gs://srinija/sample_data/sales_data_sample.csv"
val salesData = spark.read
  .format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(salesDataPath)



salesDataPath = gs://srinija/sample_data/sales_data_sample.csv
salesData = [ORDERNUMBER: int, QUANTITYORDERED: int ... 23 more fields]


[ORDERNUMBER: int, QUANTITYORDERED: int ... 23 more fields]

In [20]:
salesData.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

In [21]:
val filteredData = salesData.select(
  $"ORDERNUMBER",
  $"SALES",
  $"ORDERDATE",
  $"STATUS",
  $"CUSTOMERNAME",
  $"COUNTRY"
)
filteredData.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- COUNTRY: string (nullable = true)



filteredData = [ORDERNUMBER: int, SALES: double ... 4 more fields]


[ORDERNUMBER: int, SALES: double ... 4 more fields]

In [22]:
// val salesData = Seq(
//   ("ProductA", 100, "2024-01-01"),
//   ("ProductB", 200, "2024-01-02"),
//   ("ProductA", 300, "2024-01-03"),
//   ("ProductB", 400, "2024-01-04"),
//   ("ProductC", 500, "2024-01-05")
// ).toDF("Product", "Revenue", "Date")

Syntax Error.: 

In [23]:
def measureExecutionTime(description: String, action: => DataFrame): Unit = {
  val startTime = System.currentTimeMillis()
  action.show(5) // Show top 5 rows for quick inspection
  val endTime = System.currentTimeMillis()
  println(s"$description took ${endTime - startTime} ms")
}


println("Without caching:")
measureExecutionTime("Total sales by country", filteredData.groupBy("COUNTRY").agg(sum("SALES").as("TotalSales")))
measureExecutionTime("Filter orders with status 'Shipped'", filteredData.filter($"STATUS" === "Shipped"))

Without caching:
+-------+------------------+
|COUNTRY|        TotalSales|
+-------+------------------+
| Norway| 307463.7000000001|
|Ireland|          57756.43|
|Germany|220472.08999999994|
|Finland| 329581.9100000001|
|  Spain|1215686.9200000009|
+-------+------------------+
only showing top 5 rows

Total sales by country took 1369 ms
+-----------+-------+---------------+-------+--------------------+-------+
|ORDERNUMBER|  SALES|      ORDERDATE| STATUS|        CUSTOMERNAME|COUNTRY|
+-----------+-------+---------------+-------+--------------------+-------+
|      10107| 2871.0| 2/24/2003 0:00|Shipped|   Land of Toys Inc.|    USA|
|      10121| 2765.9|  5/7/2003 0:00|Shipped|  Reims Collectables| France|
|      10134|3884.34|  7/1/2003 0:00|Shipped|     Lyon Souveniers| France|
|      10145| 3746.7| 8/25/2003 0:00|Shipped|   Toys4GrownUps.com|    USA|
|      10159|5205.27|10/10/2003 0:00|Shipped|Corporate Gift Id...|    USA|
+-----------+-------+---------------+-------+----------------

measureExecutionTime: (description: String, action: => org.apache.spark.sql.DataFrame)Unit


In [24]:
salesData.cache()

println("\nWith caching:")
measureExecutionTime("Total sales by country (cached)", filteredData.groupBy("COUNTRY").agg(sum("SALES").as("TotalSales")))
measureExecutionTime("Filter orders with status 'Shipped' (cached)", filteredData.filter($"STATUS" === "Shipped"))


With caching:
+-------+------------------+
|COUNTRY|        TotalSales|
+-------+------------------+
| Norway| 307463.7000000001|
|Ireland|          57756.43|
|Germany|220472.08999999994|
|Finland| 329581.9100000001|
|  Spain|1215686.9200000009|
+-------+------------------+
only showing top 5 rows

Total sales by country (cached) took 2632 ms
+-----------+-------+---------------+-------+--------------------+-------+
|ORDERNUMBER|  SALES|      ORDERDATE| STATUS|        CUSTOMERNAME|COUNTRY|
+-----------+-------+---------------+-------+--------------------+-------+
|      10107| 2871.0| 2/24/2003 0:00|Shipped|   Land of Toys Inc.|    USA|
|      10121| 2765.9|  5/7/2003 0:00|Shipped|  Reims Collectables| France|
|      10134|3884.34|  7/1/2003 0:00|Shipped|     Lyon Souveniers| France|
|      10145| 3746.7| 8/25/2003 0:00|Shipped|   Toys4GrownUps.com|    USA|
|      10159|5205.27|10/10/2003 0:00|Shipped|Corporate Gift Id...|    USA|
+-----------+-------+---------------+-------+---------

In [25]:
salesData.unpersist()
spark.stop()